# Práctica: Determinación del Búfer Óptimo para Mitigar el Jitter

**Autor:** Ivelin, Samuel, Daniel, Lorenzo
**Asignatura:** Tecnologías Multimedia

## 1. Introducción y Objetivos

El objetivo de esta práctica es determinar experimentalmente el tamaño de búfer óptimo necesario para mantener una comunicación de audio fluida y de calidad bajo condiciones de red adversas simuladas.

En la comunicación de voz sobre IP (VoIP), la variabilidad en el retardo de llegada de los paquetes, conocida como **jitter**, puede degradar severamente la calidad de la experiencia (QoE), provocando que el audio se escuche entrecortado. Para combatir esto, los sistemas receptores implementan un búfer de jitter, que introduce un pequeño retardo deliberado para poder reordenar y reproducir los paquetes de audio a un ritmo constante.

Sin embargo, un búfer demasiado grande, aunque soluciona el problema del jitter, introduce una latencia excesiva que dificulta la comunicación en tiempo real. Por lo tanto, el objetivo es encontrar el **equilibrio perfecto**: el búfer más pequeño posible que elimine los artefactos del jitter sin añadir un retardo perceptible.

## 2. Fundamentos Teóricos

Para abordar el problema, es fundamental comprender tres conceptos interrelacionados:

* **Latencia**: Es el retardo total que sufre un paquete de datos desde que es enviado por el emisor hasta que es recibido por el receptor. En una conversación, una latencia alta provoca que las respuestas tarden en llegar, haciendo la interacción antinatural.

* **Jitter**: Es la **variación** de la latencia. En una red ideal, todos los paquetes tardarían lo mismo en llegar. En una red real, la congestión y otros factores provocan que los paquetes lleguen a intervalos irregulares.  El jitter es la causa directa de que el sonido se oiga con cortes, chasquidos o un "tono robótico".

* **Búfer de Jitter**: Es una pequeña memoria en el receptor que actúa como una "sala de espera" para los paquetes de audio. Su función es almacenar los paquetes por un breve instante para absorber las irregularidades causadas por el jitter, reordenarlos y entregarlos al sistema de reproducción de audio a un ritmo perfectamente constante, garantizando una experiencia auditiva fluida.

## 3. Metodología y Configuración del Experimento

Para realizar las pruebas de forma controlada y repetible, se ha optado por simular las condiciones de red en un entorno local (`localhost`). Esto nos permite aislar el efecto del búfer sin las variables incontrolables de una conexión a Internet real.

La herramienta utilizada es la utilidad de Linux `tc` (Traffic Control) con el módulo `netem` (Network Emulator). Se ha aplicado una regla a la interfaz de red de loopback (`lo`) para simular una conexión con las siguientes características:

* **Latencia media**: 80 ms
* **Jitter (variación)**: 40 ms

Esto implica que el retardo de cada paquete oscilará aleatoriamente entre **40 ms** (80 - 40) y **120 ms** (80 + 40).

El experimento consiste en ejecutar dos instancias del script `buffer.py` en local (un emisor y un receptor) y variar el tamaño del búfer (`-b` en milisegundos), analizando tanto la calidad del audio percibida como las estadísticas de red arrojadas por el programa.

In [ ]:
# Comando para aplicar la simulación de red (ejecutado en una terminal separada)
!sudo tc qdisc add dev lo root netem delay 80ms 40ms

# Comandos para iniciar la comunicación (ejecutados en dos terminales separadas)
# Terminal Receptor:
python3 buffer.py -a localhost -p 5555 -l 4444 -b 120 --show_stats
# Terminal Emisor:
python3 buffer.py -a localhost -p 4444 -l 5555 -b 120 --show_stats

: 

## 4. Resultados y Análisis

A continuación, se presentan los resultados obtenidos para diferentes tamaños de búfer.

### 4.1. Prueba con Búfer de 20 ms (`-b 20`)

**Análisis Cualitativo:**
La calidad del audio fue extremadamente pobre. El sonido se escuchaba muy entrecortado, con largos silencios y fragmentos de voz ininteligibles. Era imposible mantener una comunicación.

**Análisis Cuantitativo:**
Los logs confirman la percepción cualitativa. Existe una gran discrepancia entre los datos enviados y los recibidos, lo que indica una pérdida masiva de paquetes porque llegan más tarde de lo que el búfer de 20 ms puede esperar.

In [ ]:
# Ejemplo de salida del Receptor con buffer de 20 ms
"""
cycle  mesgs.  mesgs.   KBPS    KBPS %CPU %CPU
===============================================
    1      40       0   1265       0    1    0
    2      44       0   1442       0    0    0
    3      43       0   1409       0    0    0
    4      43       0   1409       0    0    0
    5      43      15   1409     491    0   10
...
===============================================
Avgs:      42      37   1407    1213    1    2

CPU usage average = 1.118... %
Payload sent average = 1407.63... kilo bits per second
Payload received average = 1213.33... kilo bits per second
"""

### 4.2. Prueba con Búfer de 80 ms (`-b 80`)

**Análisis Cualitativo:**
La calidad mejoró notablemente en comparación con los búferes más bajos, pero la comunicación aún no era perfecta. Se seguían percibiendo cortes y "glitches" ocasionales, especialmente en momentos de mayor congestión simulada.

**Análisis Cuantitativo:**
Los logs muestran que, aunque la diferencia entre KBPS enviados y recibidos se ha reducido, todavía existe una brecha significativa. El promedio final del receptor (`sent 1408 kbps` vs. `recv 1235 kbps`) indica que el búfer de 80 ms sigue siendo insuficiente para manejar los picos de jitter que alcanzan los 120 ms.

In [ ]:
# Ejemplo de salida del Receptor con buffer de 80 ms
"""
cycle  mesgs.  mesgs.   KBPS    KBPS %CPU %CPU
===============================================
    1      41       0   1299       0    0    0
    2      43       0   1409       0    0    0
...
    5      44      23   1441     753    0   10
    6      43      43   1409    1409    0    1
...
===============================================
Avgs:      43      37   1408    1235    0    1

CPU usage average = 0.998... %
Payload sent average = 1408.61... kilo bits per second
Payload received average = 1235.02... kilo bits per second
"""

### 4.3. Prueba con Búfer de 120 ms (`-b 120`) - El Punto Óptimo

**Análisis Cualitativo:**
Con un búfer de 120 ms, la calidad del audio se volvió **perfecta y fluida**. Desaparecieron por completo los cortes y el sonido era estable y claro. La comunicación se sentía natural, sin retardos perceptibles.

**Análisis Cuantitativo:**
Los datos respaldan esta observación de forma concluyente. Las columnas de mensajes y KBPS enviados y recibidos son prácticamente idénticas, ciclo a ciclo. El promedio final (`sent 1403 kbps` vs. `recv 1412 kbps`) muestra una paridad casi perfecta, indicando que el búfer está absorbiendo con éxito toda la variación de retardo introducida.

In [ ]:
# Ejemplo de salida del Receptor con buffer de 120 ms
"""
cycle  mesgs.  mesgs.   KBPS    KBPS %CPU %CPU
===============================================
    1      43      44   1371    1403    1    0
    2      43      42   1409    1376    0    1
    3      43      44   1409    1442    1    1
    4      43      43   1409    1409    0    2
...
===============================================
Avgs:      43      43   1403    1412    1    1

CPU usage average = 1.277... %
Payload sent average = 1403.57... kilo bits per second
Payload received average = 1412.71... kilo bits per second
"""

### 4.4. Prueba con Búfer de 150 ms (`-b 150`)

**Análisis Cualitativo:**
La calidad del audio se mantuvo perfecta y fluida, igual que con 120 ms. Sin embargo, al prestar atención, se pudo percibir un **ligero pero notable retardo** entre el momento en que se hablaba y el momento en que se escuchaba la propia voz.

**Análisis Cuantitativo:**
Los logs, al igual que en el caso anterior, muestran una comunicación estable. La diferencia clave no reside en los datos de paquetes, sino en la latencia total del sistema, que ahora incluye 150 ms de retardo artificial introducido por el búfer.

## 5. Conclusiones

Los resultados experimentales demuestran de forma clara y concluyente la función del búfer de jitter y la importancia de su correcta dimension.

Se ha determinado que, para unas condiciones de red con una latencia media de 80 ms y un jitter de 40 ms, el **búfer óptimo es de 120 ms**.

Este valor se justifica teóricamente, ya que corresponde al retardo máximo que puede experimentar un paquete en la red simulada (`80ms + 40ms = 120ms`). Un búfer de este tamaño es el mínimo necesario para garantizar que no se descarte ningún paquete por llegada tardía.

Si bien un búfer superior (como el de 150 ms) también garantiza una comunicación fluida, introduce una latencia adicional innecesaria que perjudica la interactividad de la conversación. Por lo tanto, se concluye que el valor óptimo es aquel que resuelve completamente los problemas de calidad (jitter) con el menor impacto posible en la latencia.

In [ ]:
# Comando para eliminar la regla de simulación de red y restaurar la configuración normal
!sudo tc qdisc del dev lo root

# Participantes del Proyecto

---

## 👥 Participantes

* **Ivelin Iliyanov Apostolov**
* **Samuel Mancebo Ortega**
* **Daniel García Gualda**
* **Lorenzo Valentín Cretu Abutnaritei**